ECE 5460, Au23

Solution for PyTorch Tutorial - final model only

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

ds = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# spliting training into train and validation
generator = torch.Generator().manual_seed(24)
(trainset, valset) = torch.utils.data.random_split(ds, [0.8, 0.2], generator=generator)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


We will use a convolutional network for classification.

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 80)
        self.fc2 = nn.Linear(80, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()

Define a loss function and optimizer

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001) #Experiment with momentum term and other learning rates

Define training function

In [4]:
def train_step(model, inputs, labels):
  # zero the parameter gradients
  optimizer.zero_grad()

  # forward call
  outputs = model(inputs)

  # calculate loss
  loss = criterion(outputs, labels)

  # back propagation and optimization
  loss.backward()
  optimizer.step()

  return loss

Define evaluation function

Same can be used for testing

In [5]:
def val_step(model, inputs, labels):
  # get inputs and labels from given batch
  with torch.no_grad():
    outputs = model(inputs)

    # calculate the predicted class
    _, predicted = torch.max(outputs.data, 1)
    loss = criterion(outputs, labels)
    correct_pred = (predicted == labels).sum().item() # correctly classified samples of the current batch
  return correct_pred, loss

Main training and validation

In [6]:
import time
num_epochs = 50 # change this to a larger number to train the final network


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("using device: ", device)


model = model.to(device)

val_acc_log = []
val_loss_log = []

for i in range(num_epochs):
  running_loss = 0
  for j, batch in enumerate(trainloader):
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)

    # training step
    loss = train_step(model, inputs, labels)
    running_loss += loss.item()

    # print some statistics
    if (j+1) % 1000 == 0 :
      print("epoch {}, step = {}, running loss = {:.4f}".format(i, str(j+1).zfill(5), running_loss / 1000))
      running_loss = 0



  # validation at the end of each epoch
  total_correct_pred = 0
  val_loss = []
  for _, batch in enumerate(valloader):
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)

    curr_correct_pred, curr_loss = val_step(model, inputs, labels)

    # keep track of corrected samples and loss
    total_correct_pred += curr_correct_pred
    val_loss.append(curr_loss.item())

  val_loss = torch.Tensor(val_loss).mean()
  acc = total_correct_pred / len(valset)
  print("val accuracy: {:.2f}, avg loss = {:.3f}".format(100*acc, val_loss))
  val_acc_log.append(acc * 100)
  val_loss_log.append(val_loss.item())

  # save the checkpoint
  torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 'epoch_{}_val_acc_{}.pt'.format(i, 100*acc))




using device:  cuda:0
epoch 0, step = 01000, running loss = 2.3021
epoch 0, step = 02000, running loss = 2.2958
epoch 0, step = 03000, running loss = 2.2863
epoch 0, step = 04000, running loss = 2.2621
epoch 0, step = 05000, running loss = 2.2042
epoch 0, step = 06000, running loss = 2.1492
epoch 0, step = 07000, running loss = 2.0899
epoch 0, step = 08000, running loss = 2.0751
epoch 0, step = 09000, running loss = 2.0364
epoch 0, step = 10000, running loss = 1.9918
val accuracy: 28.16, avg loss = 1.991
epoch 1, step = 01000, running loss = 1.9878
epoch 1, step = 02000, running loss = 1.9158
epoch 1, step = 03000, running loss = 1.9269
epoch 1, step = 04000, running loss = 1.8802
epoch 1, step = 05000, running loss = 1.8492
epoch 1, step = 06000, running loss = 1.8257
epoch 1, step = 07000, running loss = 1.7782
epoch 1, step = 08000, running loss = 1.7604
epoch 1, step = 09000, running loss = 1.7374
epoch 1, step = 10000, running loss = 1.7115
val accuracy: 37.41, avg loss = 1.712
ep

Testing

The other things to do is to calculate the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) to see the accuracy of individual classes.

In [8]:
from os import access
total_correct_pred = 0

for i, batch in enumerate(testloader):
  inputs, labels = batch
  inputs = inputs.to(device)
  labels = labels.to(device)

  curr_correct_pred, _ = val_step(model, inputs, labels)

  # adding up correctly predicted samples
  total_correct_pred += curr_correct_pred

# total accuracy
acc = total_correct_pred / len(testloader.dataset)
print("testing accuracy: {:.2f}%".format(100*acc))

testing accuracy: 61.88%
